In [1]:
import numpy as np
import pandas as pd
from quasinet.qnet import qdistance, load_qnet
from emergenet.domseq import DomSeq, save_model, load_model
import os
import glob

In [2]:
Q_PATH='../../paper_data_v2//irat_enet/enet_models/current_enets/h1n1_ha.joblib.gz'
qnet__=load_qnet(Q_PATH,gz=True)

In [3]:
import multiprocessing as mulpro
import itertools
import shap

In [4]:
df=pd.read_csv('../../paper_data_v2/irat_enet/results/animal_predictions/combined_results.csv')

In [5]:
df_=df[df.subtype=='H1N1'][['ha_accession','ha','emergence_risk']].set_index('ha_accession').sort_values('emergence_risk')
low_risk_seq=df_[df_.emergence_risk<6].ha.values
high_risk_seq=df_[df_.emergence_risk>6.5].ha.values

S0=[np.array(list(x)) for x in low_risk_seq]
slow=pd.DataFrame(S0).mode().values[0]
shigh=np.array([np.array(list(x)) for x in high_risk_seq])

In [6]:
s0__=slow
def fpar(s):
    return qdistance(s0__,s,qnet__,qnet__)
def f(s_array):
    pool = mulpro.Pool(processes=10)
    return np.array(pool.map(fpar, s_array))    

In [ ]:
explainer = shap.KernelExplainer(f,np.array([slow]))

In [ ]:
S_=np.array([x for x in shigh if np.random.rand()<.5 ])
shap_values = explainer.shap_values(S_, nsamples=550)

In [ ]:
shp1=pd.DataFrame(pd.DataFrame(shap_values).abs().mean().sort_values(ascending=False),columns=['shp_h1n1'])
shp1.index.name='H1N1_features'

In [ ]:
Q_PATH='../../paper_data_v2//irat_enet/enet_models/current_enets/h3n2_ha.joblib.gz'
qnet__=load_qnet(Q_PATH,gz=True)


In [ ]:
df_=df[df.subtype=='H3N2'][['ha_accession','ha','emergence_risk']].set_index('ha_accession').sort_values('emergence_risk')
low_risk_seq=df_[df_.emergence_risk<6].ha.values
high_risk_seq=df_[df_.emergence_risk>6.5].ha.values

S0=[np.array(list(x)) for x in low_risk_seq]
slow=pd.DataFrame(S0).mode().values[0]
shigh=np.array([np.array(list(x)) for x in high_risk_seq])

In [ ]:
s0__=slow
def fpar(s):
    return qdistance(s0__,s,qnet__,qnet__)
def f(s_array):
    pool = mulpro.Pool(processes=10)
    return np.array(pool.map(fpar, s_array))    

In [ ]:
explainer = shap.KernelExplainer(f,np.array([slow]))

In [ ]:
S_=np.array([x for x in shigh if np.random.rand()<.5 ])
shap_values = explainer.shap_values(S_, nsamples=550)

In [ ]:
shp2=pd.DataFrame(pd.DataFrame(shap_values).abs().mean().sort_values(ascending=False),columns=['shp_h3n2'])
shp2.index.name='H3N2_features'

In [ ]:
shp1.to_csv('lowhighrisk_H1N1.csv')
shp2.to_csv('lowhighrisk_H3N2.csv')


In [ ]:
ax=shp2.sort_index().ewm(alpha=.95).mean().plot(logy=False)
shp1.sort_index().ewm(alpha=.95).mean().plot(logy=False,ax=ax)


In [ ]:
shp1=pd.read_csv('lowhighrisk_H1N1.csv',index_col=0)
shp2=pd.read_csv('lowhighrisk_H3N2.csv',index_col=0)


In [ ]:
shp1.head(10)

In [ ]:
s1=shp1.sort_index().ewm(alpha=.85).mean()
s1=s1/s1.max()
s2=shp2.sort_index().ewm(alpha=.85).mean()
s2=s2/s2.max()
ax=s2.plot(logy=False)
#ax.set_ylim(0,0.005)
s1.plot(logy=False,ax=ax)


In [ ]:
s1

In [ ]:
s2[s2>0.05].dropna().index.values

In [ ]:
' '.join(s1[s1>0.05].dropna().index.values.astype(str))

In [ ]:
' '.join(s1[s1.shp_h1n1.between(0.01,0.05)].dropna().index.values.astype(str))

In [ ]:
! ls *SHAP*csv


In [ ]:
sf_n_h1n1=pd.read_csv('north_h1n1_ha_SHAP.csv',index_col=0)
sf_n_h1n1=sf_n_h1n1.sort_index().ewm(alpha=.85).mean()
sf_n_h1n1=sf_n_h1n1/sf_n_h1n1.max()
sf_n_h1n1[sf_n_h1n1>0.03].dropna().index.values

In [ ]:
sf_n_h1n1.plot()

In [ ]:
sf_s_h1n1=pd.read_csv('south_h1n1_ha_SHAP.csv',index_col=0)
sf_s_h1n1=sf_s_h1n1.sort_index().ewm(alpha=.85).mean()
sf_s_h1n1=sf_s_h1n1/sf_s_h1n1.max()
sf_s_h1n1[sf_s_h1n1>0.03].dropna().index.values

In [ ]:
sf_s_h1n1.plot()